In [ ]:
# Install a pip package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install --force-reinstall hive_api-0.0.4-py3-none-any.whl
!{sys.executable} -m pip install langchain-ollama

In [ ]:
from hive_api import HiveApi
hive = HiveApi()

from hive_agent import HiveAgent, AgentState
agent = HiveAgent(hive, debug_graph=False)

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage

class ConversationGraph():
    def __init__(self, ollama_url="http://ollama:11434", ollama_model="mistral:latest"):
        self._node_name = "conversation_node"
        self._json_model = OllamaLLM(base_url=ollama_url, model=ollama_model, temperature=0.7)
        self._prompt = PromptTemplate.from_template("""You are a smart assistant.
Answer if you know else tell the user you can not provide any information and describe how to find the answer.

{conversation}

""")
        self._chain = self._prompt | self._json_model

    def node_name(self):
        return self._node_name

    def chain(self):
        return self._chain

    def node(self, state: AgentState, config):
        """ Answer the user message """
        print("{}.node() called".format( self._node_name))
        state["messages"] = AIMessage(content=self._chain.invoke({"conversation": state["messages"]}))
        return state
    
    def edge(self, state: AgentState, config):
        """ Nothing special here """
        return state


class SubGraph():
    def __init__(self, hive):
        self.name = "SubGraph.workflow()"
        self.conversation = ConversationGraph(ollama_url=hive.get_api_url("ollama"))
        self.conversation2 = ConversationGraph(ollama_url=hive.get_api_url("ollama"))

    def debug_node(self, state: AgentState, config):
        user = config["configurable"].get("user", "unknown")
        print("SubGraph.debug_node() User {} request".format(user))
        return state
    
    def workflow(self, subgraph=None, subname="debug_node", memory=None):
        # Define a new graph
        workflow = StateGraph(AgentState)
        subnode = self.debug_node
        if subgraph:
            subnode = subgraph
            if subname == "debug_node":
                subname = "subgraph"

        # Define the two nodes we will cycle between
        workflow.add_node("SubGraph.conversation", self.conversation.node)
        workflow.add_node("SubGraph.conversation2", self.conversation2.node)
        workflow.add_node("SubGraph.{}".format(subname), subnode)
        workflow.set_entry_point("SubGraph.conversation")

        workflow.add_edge("SubGraph.conversation", "SubGraph.{}".format(subname))
        workflow.add_edge("SubGraph.{}".format(subname), "SubGraph.conversation2")
        workflow.add_edge("SubGraph.conversation2", END)
        if memory:
            return workflow.compile(checkpointer=memory)
        else:
            return workflow.compile()


subgraph = SubGraph(hive)
agent.build_graph(subgraph.workflow(memory=agent.get_memory()))
agent.show_graph(True)

In [ ]:
for event in agent.on_input("bonjour"):
    print(event)

In [ ]:
for event in agent.on_input("alors ça donne quoi"):
    print(event)